<a href="https://colab.research.google.com/github/Sakib-Khandaker/E-Commerce-Website-Super-Shop/blob/main/Project_Machin_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sakib Khandakar\
Batch: 18

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
train_dir = "/content/drive/MyDrive/Project_Photos/train"
val_dir = "/content/drive/MyDrive/Project_Photos/test"

In [ ]:
train_transforms = transforms.Compose([
    transforms.CenterCrop((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transforms = transforms.Compose([
    transforms.CenterCrop((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
val_dataset = datasets.ImageFolder(val_dir, transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, 12)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 32 * 32)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

SimpleCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=65536, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=12, bias=True)
  (relu): ReLU()
)

In [ ]:
num_epochs = 10
iteration = 0

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    print("-" * 20)

    model.train()
    running_loss = 0.0
    correct = 0
    total = 0


    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        iteration = iteration + 1


        optimizer.zero_grad()


        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item():.4f}")

        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct / total
    print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

torch.save(model.state_dict(), "custom_image_classifier.pth")
print("Model training complete and saved.")


Epoch 1/10
--------------------
Loss: 2.4747
Loss: 1.4940
Loss: 4.7204
Loss: 5.1322
Loss: 4.9727
Loss: 1.9748
Loss: 1.8665
Train Loss: 3.4132 Acc: 0.3333
Epoch 2/10
--------------------
Loss: 1.2773
Loss: 1.8461
Loss: 1.5680
Loss: 1.7233
Loss: 1.7807
Loss: 1.6309
Loss: 0.9423
Train Loss: 1.6167 Acc: 0.3737
Epoch 3/10
--------------------
Loss: 1.2047
Loss: 1.3416
Loss: 1.1529
Loss: 1.6139
Loss: 1.4164
Loss: 1.6261
Loss: 1.7821
Train Loss: 1.4044 Acc: 0.5051
Epoch 4/10
--------------------
Loss: 1.1575
Loss: 1.7426
Loss: 0.9534
Loss: 0.5058
Loss: 1.5793
Loss: 1.1535
Loss: 0.4992
Train Loss: 1.1614 Acc: 0.5657
Epoch 5/10
--------------------
Loss: 0.8175
Loss: 0.9651
Loss: 1.0890
Loss: 1.0067
Loss: 0.6545
Loss: 0.9274
Loss: 1.0275
Train Loss: 0.9136 Acc: 0.6263
Epoch 6/10
--------------------
Loss: 0.8590
Loss: 0.8609
Loss: 0.9339
Loss: 0.8877
Loss: 1.0396
Loss: 0.7368
Loss: 0.5101
Train Loss: 0.8749 Acc: 0.6667
Epoch 7/10
--------------------
Loss: 0.4331
Loss: 0.9166
Loss: 0.7422
Loss:

In [ ]:
torch.save(model.state_dict(), "custom_image_classifier.pth")
print("Model training complete and saved.")


Model training complete and saved.


In [ ]:
model.eval()
val_loss = 0.0
val_correct = 0
val_total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        val_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        val_total += labels.size(0)
        val_correct += (predicted == labels).sum().item()

val_epoch_loss = val_loss / len(val_dataset)
val_epoch_acc = val_correct / val_total
print(f"Val Loss: {val_epoch_loss:.4f} Acc: {val_epoch_acc:.4f}")

Val Loss: 1.5025 Acc: 0.5000
